# Update Datasets

Outline:

- PreReq
- Ticker List
- Static Data:
-- Quotes
-- Description
-- Fundamental
- Live Data:
-- Intraday Quotes
-- News

## Pre-Requisites

### G-Drive Connect
Authorise google colaboratory to access drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import clear_output 
clear_output()

### Requirements

In [2]:
!pip install yfinance finvizfinance finviz finpie yahoo_fin
clear_output()

### Import dependencies

In [3]:
import pandas as pd
import finpie
import yfinance as yf
import finviz
import finvizfinance
import time
import progressbar
from finvizfinance.screener.overview import Overview
from yahoo_fin import stock_info as si
import requests
from pandas_datareader import data as pdr
import warnings
warnings.filterwarnings("ignore")
clear_output()

## Static Data

The static data records the historic price values for the equities. This should be updated after the markets close at 8 pm EST.

### Ticker List
The ticker symbol list needs to be updated as new symbols are being listed/delisted everyday.

In [4]:
def update_ticker_list():
  # Download latest ticker list from Nasdaq
  ticker_list=finpie.nasdaq_tickers()
  # Select Symbol Column
  ticker_list=ticker_list['Symbol']
  # Save ticker list to csv
  ticker_list.to_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv',index=None)
  # Download latest ticker list in S&P500
  sp500_ticker_list = si.tickers_sp500(include_company_data=True)
  # Select Symbol Column
  sp500_ticker_list=sp500_ticker_list['Symbol']
  # Save ticker list to csv
  sp500_ticker_list.to_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/S&P500.csv',index=None)

### Stock Data

The download functionality takes about 3 hours to completely download historic data for all 9000+ symbols. Modify your serach criteria for downloading datasets to save time.

In [5]:
def set_status(ticker_number,fname):
  ticker_number=str(ticker_number)
  with open(f'/content/drive/My Drive/Master Project/Dataset/logs/{fname}_data_update_stat.txt', 'w') as writefile:
      writefile.write(ticker_number)

In [6]:
def get_status(fname):
  ticker_number=0
  try:
    with open(f'/content/drive/My Drive/Master Project/Dataset/logs/{fname}_data_update_stat.txt', 'r') as logfile:
      ticker_number=int(logfile.read())
  except:
    pass
  return ticker_number

In [7]:
def fetching_news(ticker,url="https://api.queryly.com/cnbc/json.aspx"):
  #stock_fundamentals=finpie.Fundamentals(ticker)
  #stock_profile=stock_fundamentals.profile()
  #company=stock_profile['company_name'][0]
  #ticker="AAPL"
  #company="Apple Inc"
  #search_query=company+" "+ticker
  #search_query=company
  search_query=ticker
  params = {
      "queryly_key": "31a35d40a9a64ab3",
      "query": search_query,
      "endindex": "0",
      "batchsize": "100",
      "callback": "",
      "showfaceted": "true",
      "timezoneoffset": "-120",
      "facetedfields": "formats",
      "facetedkey": "formats|",
      "facetedvalue":
      "!Press Release|",
      "needtoptickers": "1",
      "additionalindexes": "4cd6f71fbf22424d,937d600b0d0d4e23,3bfbe40caee7443e,626fdfcd96444f28"
  }
  header = ["cn:title", "_pubDate", "cn:liveURL", "description"]
  with requests.Session() as req:
      allin = []
      for page, item in enumerate(range(0, 100000, 100)):
          #print(f"Extracting Page# {page +1}")
          params["endindex"] = item
          r = req.get(url, params=params).json()
          empty=True
          for loop in r['results']:
            allin.append([loop[x] for x in header])
            empty=False
          if empty:
            break
  result = pd.DataFrame(allin, columns=["Title", "Date", "Url", "Description"])
  result.to_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/news_data/{}.csv'.format(ticker),index=False)
  return result

In [8]:
def fetch_option_chain(ticker):
  calls,puts=finpie.yahoo_option_chain(ticker)
  calls.to_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/option_chain/{}_calls.csv'.format(ticker))
  puts.to_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/option_chain/{}_puts.csv'.format(ticker))

In [9]:
def fetch_historic_price(ticker):
  data=finpie.historical_prices(ticker)
  data.to_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/market_historic/{}.csv'.format(ticker))

In [10]:
def fetch_ticker_profile(ticker):
  stock=finpie.Fundamentals(ticker)
  data=stock.profile().join(stock.key_metrics())
  data.to_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/stock_profile/{}.csv'.format(ticker))

In [11]:
def update_historical_data(letter="sp500"):
  symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv')
  if letter == "all":
    symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv')
  elif letter == "sp500":
    symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/S&P500.csv')
  else:
    symbols=symbols[[x.startswith(letter) for x in symbols['Symbol']]]

  bar = progressbar.ProgressBar(
      widgets=[
          progressbar.Counter(format='%(value)5d/%(max_value)d | %(dynamic_messages)6s | '),
          progressbar.Timer(format= 'Elapsed: %(elapsed)s '),
          progressbar.Bar('*'),
          ' ',
          progressbar.AdaptiveETA(),
      ],
  )
  log_start=get_status(letter)
  shift=0
  for symbol in bar(symbols['Symbol'][log_start:]):
    try:
      bar.dynamic_messages=symbol
      fetch_ticker_profile(symbol)
      fetch_historic_price(symbol)
      fetch_option_chain(symbol)
      fetching_news(symbol)
      shift+=1
      val=(log_start+shift)%symbols.last_valid_index()
      set_status(val,letter)
    except:
      pass
  set_status(0,letter)

In [12]:
'''
msft = yf.Ticker("MSFT")
# get stock info
msft.info
# get historical market data
hist = msft.history(period="max")
# show actions (dividends, splits)
msft.actions
# show dividends
msft.dividends
# show splits
msft.splits
# show financials
msft.financials
msft.quarterly_financials
# show major holders
msft.major_holders
# show institutional holders
msft.institutional_holders
# show balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet
# show cashflow
msft.cashflow
msft.quarterly_cashflow
# show earnings
msft.earnings
msft.quarterly_earnings
# show sustainability
msft.sustainability
# show analysts recommendations
msft.recommendations
# show next event (earnings, etc)
msft.calendar
# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
#msft.isin
# show options expirations
#options_dates=msft.options
# get option chain for specific expiration
#opt = msft.option_chain('2021-03-19')
# data available via: opt.calls, opt.puts
#opt.calls
'''
clear_output()

## Latest Intraday (2 years)

In [13]:
def fetch_intraday_price(data,symbols):
  list_type="intraday"
  bar = progressbar.ProgressBar(
      widgets=[
          progressbar.Counter(format='Saving to File | %(value)5d/%(max_value)d | %(dynamic_messages)6s | '),
          progressbar.Timer(format= 'Elapsed: %(elapsed)s '),
          progressbar.Bar('*'),
          ' ',
          progressbar.AdaptiveETA(),
      ],
  )
  log_start=get_status(list_type)
  shift=0
  for symbol in bar(symbols['Symbol'][log_start:]):
    try:
      bar.dynamic_messages=symbol
      stock=data[symbol]
      stock.dropna(inplace=True)
      stock.to_csv(f'/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/market_intraday/{symbol}.csv')
      shift+=1
      val=(log_start+shift)%symbols.last_valid_index()
      set_status(val,letter)
    except:
      pass
  set_status(0,list_type)
  clear_output()

In [14]:
def update_intraday_master(letter="sp500"):
  symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv')
  if letter == "all":
    symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv')
  elif letter == "sp500":
    symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/S&P500.csv')
  else:
    symbols=symbols[[x.startswith(letter) for x in symbols['Symbol']]]
  stock_list=""
  i=1
  for x in symbols['Symbol']:
    if i==1:
      stock_list+=x
      i=2
    else:
      stock_list+=" "+x  
  yf.pdr_override()
  data=pdr.get_data_yahoo(stock_list,interval='60m',period='2y',group_by='ticker')
  data.to_csv(f'/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/market_intraday/__master_dataset_{letter}__.csv')
  clear_output()
  fetch_intraday_price(data,symbols)

In [15]:
#update_intraday_master()

In [16]:
#data['AAPL'][data['AAPL']['Volume']<10000]

In [17]:
symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/S&P500.csv')
stock_list=""
i=1
for x in symbols['Symbol']:
  if i==1:
    stock_list+=x
    i=2
  else:
    stock_list+=" "+x  

In [18]:
#data=yf.download(stock_list,interval='60m',period='2y',group_by='ticker')
#data=pdr.get_data_yahoo(stock_list,period='2y',group_by='ticker')

In [19]:
#data.to_csv(f'/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/market_historic/__master_dataset_sp500__.csv')

# Main

In [20]:
#update_ticker_list()

In [21]:
#update_intraday_master()

In [22]:
#update_historical_data()

In [34]:
#print("Sample News Data")
#fetching_news("GME")

,Title,Date,Url,Description
0,"Given GME volatility, size investments accordi...",2/25/2021 10:33:01 PM,https://www.cnbc.com/video/2021/02/25/given-gm...,"CNBC's ""Closing Bell"" team breaks down GameSto..."
1,Reddit trader 'Roaring Kitty' explains his GME...,2/18/2021 9:33:21 PM,https://www.cnbc.com/video/2021/02/18/reddit-t...,The House Committee on Financial Services hold...
2,Here's the difference between Reddit-fueled GM...,2/11/2021 10:34:13 PM,https://www.cnbc.com/video/2021/02/11/heres-th...,"CNBC's ""Closing Bell"" team discusses Reddit tr..."
3,Derivative strategist breaks down GME stock op...,2/2/2021 8:29:05 PM,https://www.cnbc.com/video/2021/02/02/derivati...,CNBC's Brian Sullivan talks with Chris Murphy ...
4,"GME, AMC and silver trades remain volatile whi...",2/1/2021 5:43:47 PM,https://www.cnbc.com/video/2021/02/01/gme-amc-...,CNBC's Bob Pisani reports on the market's morn...
...,...,...,...,...
910,No. 3 - Summer Stocks: Off To Camp,6/19/2007 3:35:04 AM,https://www.cnbc.com/id/19294116,"As the kids go off to camp this summer, there’..."
911,"The Word on Merger Mania, Specialty Retail and...",5/26/2007 2:16:53 AM,https://www.cnbc.com/id/18866000,"Day Off, Deals On?: The headline: Will The Lon..."
912,"Silly Dylan, ""Picks"" Are For Kids!",4/27/2007 3:01:20 AM,https://www.cnbc.com/id/18337627,It seems like traders are getting younger and ...
913,"The Word on Tiffany's, GameStop & More...",3/27/2007 3:16:42 AM,https://www.cnbc.com/id/17803172,DIAMOND IN THE ROUGH:The news: Upscale jewelry...


In [58]:
main=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/market_historic/GME.csv')
news=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/news_data/GME.csv')

In [59]:
main

,date,open,high,low,close,adj_close,volume
0,2002-02-13,9.625000,10.060000,9.525000,10.050000,6.766666,19054000
1,2002-02-14,10.175000,10.195000,9.925000,10.000000,6.733003,2755400
2,2002-02-15,10.000000,10.025000,9.850000,9.950000,6.699336,2097400
3,2002-02-19,9.900000,9.900000,9.375000,9.550000,6.430017,1852600
4,2002-02-20,9.600000,9.875000,9.525000,9.875000,6.648838,1723200
...,...,...,...,...,...,...,...
4787,2021-02-19,41.279999,43.889999,38.500000,40.590000,40.590000,14678700
4788,2021-02-22,46.689999,48.509998,42.400002,46.000000,46.000000,19338400
4789,2021-02-23,44.970001,46.230000,40.000000,44.970001,44.970001,7516000
4790,2021-02-24,44.700001,91.709999,44.700001,91.709999,91.709999,76315700


In [72]:
main=main.set_index('date')

In [73]:
main

,open,high,low,close,adj_close,volume
date,,,,,,
2002-02-13,9.625000,10.060000,9.525000,10.050000,6.766666,19054000
2002-02-14,10.175000,10.195000,9.925000,10.000000,6.733003,2755400
2002-02-15,10.000000,10.025000,9.850000,9.950000,6.699336,2097400
2002-02-19,9.900000,9.900000,9.375000,9.550000,6.430017,1852600
2002-02-20,9.600000,9.875000,9.525000,9.875000,6.648838,1723200
...,...,...,...,...,...,...
2021-02-19,41.279999,43.889999,38.500000,40.590000,40.590000,14678700
2021-02-22,46.689999,48.509998,42.400002,46.000000,46.000000,19338400
2021-02-23,44.970001,46.230000,40.000000,44.970001,44.970001,7516000


In [62]:
news

,Title,Date,Url,Description
0,"Given GME volatility, size investments accordi...",2021-02-25,https://www.cnbc.com/video/2021/02/25/given-gm...,"CNBC's ""Closing Bell"" team breaks down GameSto..."
1,Reddit trader 'Roaring Kitty' explains his GME...,2021-02-18,https://www.cnbc.com/video/2021/02/18/reddit-t...,The House Committee on Financial Services hold...
2,Here's the difference between Reddit-fueled GM...,2021-02-11,https://www.cnbc.com/video/2021/02/11/heres-th...,"CNBC's ""Closing Bell"" team discusses Reddit tr..."
3,Derivative strategist breaks down GME stock op...,2021-02-02,https://www.cnbc.com/video/2021/02/02/derivati...,CNBC's Brian Sullivan talks with Chris Murphy ...
4,"GME, AMC and silver trades remain volatile whi...",2021-02-01,https://www.cnbc.com/video/2021/02/01/gme-amc-...,CNBC's Bob Pisani reports on the market's morn...
...,...,...,...,...
910,No. 3 - Summer Stocks: Off To Camp,2007-06-19,https://www.cnbc.com/id/19294116,"As the kids go off to camp this summer, there’..."
911,"The Word on Merger Mania, Specialty Retail and...",2007-05-26,https://www.cnbc.com/id/18866000,"Day Off, Deals On?: The headline: Will The Lon..."
912,"Silly Dylan, ""Picks"" Are For Kids!",2007-04-27,https://www.cnbc.com/id/18337627,It seems like traders are getting younger and ...
913,"The Word on Tiffany's, GameStop & More...",2007-03-27,https://www.cnbc.com/id/17803172,DIAMOND IN THE ROUGH:The news: Upscale jewelry...


In [64]:
news['Date']=news['Date'].astype('datetime64[D]')
news=news.drop(columns=["Url","Description"])

In [65]:
news=news.rename(columns={"Date":"date","Title":"headline"})

In [69]:
news=news.groupby('date').agg({'headline': ' '.join})

In [70]:
news

,headline
date,
2007-03-27,"The Word on Tiffany's, GameStop & More..."
2007-03-28,Stock Updates: One Popular Pick May Have Run C...
2007-04-12,No. 3 - Wii Are The World
2007-04-27,"Silly Dylan, ""Picks"" Are For Kids!"
2007-05-24,Your First Move For Thursday May 24th
...,...
2021-02-22,These are the most shorted stocks in the U.S. ...
2021-02-24,GameStop shares soar more than 100% amid execu...
2021-02-25,"Given GME volatility, size investments accordi..."


In [75]:
combine=main.join(news,sort=True)

In [76]:
combine

,open,high,low,close,adj_close,volume,headline
date,,,,,,,
2002-02-13,9.625000,10.060000,9.525000,10.050000,6.766666,19054000,NaN
2002-02-14,10.175000,10.195000,9.925000,10.000000,6.733003,2755400,NaN
2002-02-15,10.000000,10.025000,9.850000,9.950000,6.699336,2097400,NaN
2002-02-19,9.900000,9.900000,9.375000,9.550000,6.430017,1852600,NaN
2002-02-20,9.600000,9.875000,9.525000,9.875000,6.648838,1723200,NaN
...,...,...,...,...,...,...,...
2021-02-19,41.279999,43.889999,38.500000,40.590000,40.590000,14678700,NaN
2021-02-22,46.689999,48.509998,42.400002,46.000000,46.000000,19338400,These are the most shorted stocks in the U.S. ...
2021-02-23,44.970001,46.230000,40.000000,44.970001,44.970001,7516000,NaN


In [77]:
combine=combine.dropna()

In [78]:
combine

,open,high,low,close,adj_close,volume,headline
date,,,,,,,
2007-03-27,30.660000,31.430000,30.250000,31.160000,20.980032,9239200,"The Word on Tiffany's, GameStop & More..."
2007-03-28,31.120001,32.980000,30.930000,32.599998,21.949587,5201400,Stock Updates: One Popular Pick May Have Run C...
2007-04-12,34.340000,34.389999,33.599998,33.910000,22.831612,1472400,No. 3 - Wii Are The World
2007-04-27,34.040001,34.080002,33.470001,33.610001,22.629618,2469200,"Silly Dylan, ""Picks"" Are For Kids!"
2007-05-24,35.869999,36.250000,34.790001,35.209999,23.706896,3187200,Your First Move For Thursday May 24th
...,...,...,...,...,...,...,...
2021-02-11,50.009998,55.320000,48.220001,51.099998,51.099998,12997400,Here's the difference between Reddit-fueled GM...
2021-02-18,48.490002,48.869999,40.650002,40.689999,40.689999,23916900,Reddit trader 'Roaring Kitty' explains his GME...
2021-02-22,46.689999,48.509998,42.400002,46.000000,46.000000,19338400,These are the most shorted stocks in the U.S. ...


# References

https://github.com/ranaroussi/yfinance/tree/master/yfinance

https://github.com/peterlacour/finpie

https://github.com/mariostoev/finviz

https://github.com/lit26/finvizfinance

https://stackoverflow.com/questions/61154530/calling-back-end-api-of-cnbc-in-python

https://github.com/wilsonfreitas/awesome-quant#python

In [42]:
#df=fetching_news("S&P 500","https://api.queryly.com/cnbc/json.aspx")
#df

In [43]:
#sp500_ticker_list = si.tickers_sp500(include_company_data=True)
#sp500_ticker_list

In [44]:
#all=si.tickers_nasdaq(include_company_data=True)
#all

In [45]:
#pandas.concat([df1,df2]).drop_duplicates().reset_index(drop=True)